<a href="https://colab.research.google.com/github/jeawsdrfgt/Anomaly-Detection/blob/main/Anomaly_detection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


Importing the python libraries needed

In [ ]:
# Generate synthetic normal and anomaly data
normal_data = np.random.normal(loc=0, scale=1, size=(1000, 20))
anomaly_data = np.random.normal(loc=5, scale=1, size=(100, 20))

# Combine the data
X = np.concatenate([normal_data, anomaly_data], axis=0)
y = np.concatenate([np.zeros(1000), np.ones(100)], axis=0)  # 0 for normal, 1 for anomaly

# Scale the data between 0 and 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


Loading and preprocessing the data

In [ ]:
# Define the autoencoder model
input_dim = X_train.shape[1]

autoencoder = models.Sequential([
    layers.InputLayer(input_shape=(input_dim,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(input_dim, activation='sigmoid')
])

# Compile the model
autoencoder.compile(optimizer='adam', loss='mse')

# Print model summary
autoencoder.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16)                  │             336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8)                   │             136 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │             144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 20)                  │             340 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 956 (3.73 KB)

 Trainable params: 956 (3.73 KB)

 Non-trainable params: 0 (0.00 B)

The Autoencoder

In [ ]:
# Train the autoencoder only on the normal data (without anomalies)
history = autoencoder.fit(X_train[y_train == 0], X_train[y_train == 0],
                          epochs=100,
                          batch_size=32,
                          validation_split=0.1,
                          verbose=1)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0066 - val_loss: 0.0068
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0065 - val_loss: 0.0068
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0067 - val_loss: 0.0068
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0065 - val_loss: 0.0068
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0066 - val_loss: 0.0068
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0066 - val_loss: 0.0068
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0065 - val_loss: 0.0068
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0066 - val_loss: 0.0068
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0065 - val_loss: 0.0068
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0068 - val_loss: 0.0068
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0065 - val_loss: 0.0068
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0

Train The model

In [ ]:
# Get reconstruction loss on test data
reconstructions = autoencoder.predict(X_test)
reconstruction_loss = tf.keras.losses.mse(X_test, reconstructions)

# Set a threshold for anomaly detection (can be adjusted based on domain knowledge)
threshold = np.mean(reconstruction_loss) + 3 * np.std(reconstruction_loss)

# Detect anomalies
anomaly_predictions = reconstruction_loss > threshold


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


Detecting Anomalies

In [ ]:
from sklearn.metrics import classification_report

# Evaluate predictions
print(classification_report(y_test, anomaly_predictions))


              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       204
         1.0       1.00      0.88      0.93        16

    accuracy                           0.99       220
   macro avg       1.00      0.94      0.96       220
weighted avg       0.99      0.99      0.99       220



Evaluating the model